In [ ]:
#librerias
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import inspect

In [ ]:
#Coneción con la base de datos a usar
engine = create_engine('sqlite:///chinook.db')
engine

Engine(sqlite:///chinook.db)

# Extract

In [ ]:
#tabla de Customers 
metadata = MetaData()
metadata.create_all(engine)
inspector = inspect(engine)
ins = inspector.get_columns('customers')

# Transform

In [ ]:
#tabla Customers
with engine.connect() as con:
    
    rs = con.execute('SELECT * FROM customers')
    
    for row in rs:
        print(row)
con.close()



(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3)
(2, 'Leonie', 'Köhler', None, 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', '+49 0711 2842222', None, 'leonekohler@surfeu.de', 5)
(3, 'François', 'Tremblay', None, '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', '+1 (514) 721-4711', None, 'ftremblay@gmail.com', 3)
(4, 'Bjørn', 'Hansen', None, 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', '+47 22 44 22 22', None, 'bjorn.hansen@yahoo.no', 4)
(5, 'František', 'Wichterlová', 'JetBrains s.r.o.', 'Klanova 9/506', 'Prague', None, 'Czech Republic', '14700', '+420 2 4172 5555', '+420 2 4172 5555', 'frantisekw@jetbrains.com', 4)
(6, 'Helena', 'Holý', None, 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', '+420 2 4177 0449', None, 'hholy@gmail.com', 5)
(7, 'Astrid', '

# Load

In [ ]:
#Realizar una consulta que devuelva el nombre del artista y el recuento total de pistas de las 10 mejores bandas de rock.
import pandas as pd

df1 = pd.read_sql_query("""SELECT 
  ar.ArtistId AS "ID Artista", 
  ar.Name AS "Artista", 
  COUNT(tr.Name) AS "Total de Pistas" 
FROM 
  artists ar 
  JOIN albums al ON al.ArtistId = ar.ArtistId 
  JOIN tracks tr ON al.AlbumId = tr.AlbumId 
  JOIN genres ge ON tr.GenreId = ge.GenreId 
WHERE 
  ge.Name = 'Rock' 
GROUP BY 
  ar.ArtistId, 
  ar.Name, 
  ge.Name 
ORDER BY 
  COUNT(tr.Name) DESC 
LIMIT 
  10;""", 
                       con=engine.connect())
df1.head()

,ID Artista,Artista,Total de Pistas
0,22,Led Zeppelin,114
1,150,U2,112
2,58,Deep Purple,92
3,90,Iron Maiden,81
4,118,Pearl Jam,54


In [ ]:
#¿Qué artistas han ganado más según las lineas de facturas (InvoiceLines)?
import pandas as pd

df2 = pd.read_sql_query("""SELECT 
  ar.Name AS "Nombre artista", ge.Name AS "Género", 
  SUM(il.Quantity * il.UnitPrice) AS "Total de compra" 
FROM 
  artists ar 
  JOIN albums al ON ar.ArtistId = al.ArtistId 
  JOIN tracks tr ON al.AlbumId = tr.AlbumId
  JOIN genres ge ON tr.AlbumId = ge.GenreId  
  JOIN invoice_items il ON tr.TrackId = il.TrackId 
  JOIN invoices i ON il.InvoiceId = i.InvoiceId 
  JOIN customers c ON i.CustomerId = c.CustomerId 
GROUP BY 
  ar.Name 
ORDER BY 
  SUM(il.Quantity * il.UnitPrice) DESC 
LIMIT 
  10;""", 
                       con=engine.connect())
df2.head()

,Nombre artista,Género,Total de compra
0,Chico Buarque,Alternative,26.73
1,Chico Science & Nação Zumbi,Opera,24.75
2,Caetano Veloso,Comedy,20.79
3,AC/DC,Alternative & Punk,15.84
4,Bruce Dickinson,TV Shows,11.88


In [ ]:
#Realizar una consulta que determine el cliente que más ha gastado en música por cada país.
import pandas as pd

df3 = pd.read_sql_query("""WITH t1 AS (
  SELECT 
    c.FirstName AS "Nombre", 
    c.LastName AS "Apellido", 
    c.Country País, 
    SUM(i.Total) TotalGastado 
  FROM 
    customers c 
    JOIN invoices i ON c.CustomerId = i.CustomerId 
  GROUP BY 
    c.FirstName
) 
SELECT 
  t1.* 
FROM 
  t1 
  JOIN(
    SELECT 
      País, 
      MAX(TotalGastado) AS MaxTotalGastado 
    FROM 
      t1 
    GROUP BY 
      País
  ) t2 ON t1.País = t2.País 
WHERE 
  t1.TotalGastado = t2.MaxTotalGastado 
ORDER BY 
  País""", 
                       con=engine.connect())
df3.head()


,Nombre,Apellido,País,TotalGastado
0,Diego,Gutiérrez,Argentina,37.62
1,Mark,Taylor,Australia,75.24
2,Astrid,Gruber,Austria,42.62
3,Daan,Peeters,Belgium,37.62
4,Luís,Gonçalves,Brazil,39.62


In [ ]:
#¿Cuál es el género musical más popular de cada país?
import pandas as pd
df4 = pd.read_sql_query("""WITH t1 AS (
  SELECT 
    c.Country País, 
    ge.Name Género, 
    COUNT(i.InvoiceId) TotalCompras 
  FROM 
    invoices i 
    JOIN customers c ON i.CustomerId = c.CustomerId 
    JOIN invoice_items il ON i.InvoiceId = il.InvoiceId 
    JOIN tracks tr ON il.TrackId = tr.TrackId 
    JOIN genres ge ON tr.GenreId = ge.GenreId 
  GROUP BY 
    País, 
    Género 
  ORDER BY 
    País, 
    COUNT(i.InvoiceId) DESC
) 
SELECT 
  t1.* 
FROM 
  t1 
  JOIN (
    SELECT 
      MAX(TotalCompras) AS MaxCompras, 
      País 
    FROM 
      t1 
    GROUP BY 
      País
  ) t2 ON t1.País = t2.País 
WHERE 
  t1.TotalCompras = t2.MaxCompras;""", 
                       con=engine.connect())
df4.head()

,País,Género,TotalCompras
0,Argentina,Alternative & Punk,9
1,Argentina,Rock,9
2,Australia,Rock,22
3,Austria,Rock,15
4,Belgium,Rock,21


In [ ]:
#Realizar una consulta que determine el cliente que más ha gastado en música por cada país.
import pandas as pd
df5 = pd.read_sql_query("""
SELECT DISTINCT
       c.FirstName AS "Nombre",
       c.LastName  AS "Apellido",
	   c.Email     AS "Correo electrónico",
       ge.Name      AS "Nombre de género"
  FROM customers c JOIN invoices i 
    ON c.CustomerId = i.CustomerId JOIN invoice_items il
    ON i.InvoiceId = il.InvoiceId JOIN tracks tr 
    ON il.TrackId = tr.TrackId JOIN genres ge 
    ON tr.GenreId = ge.GenreId 
 WHERE ge.Name = "Rock" 
ORDER BY c.Email;""", 
                       con=engine.connect())
df5.head()

,Nombre,Apellido,Correo electrónico,Nombre de género
0,Aaron,Mitchell,aaronmitchell@yahoo.ca,Rock
1,Alexandre,Rocha,alero@uol.com.br,Rock
2,Astrid,Gruber,astrid.gruber@apple.at,Rock
3,Bjørn,Hansen,bjorn.hansen@yahoo.no,Rock
4,Camille,Bernard,camille.bernard@yahoo.fr,Rock


In [ ]:
#Exportar a fuente de datos destino
df1.to_csv('dataframe1.csv', index=False)
df2.to_csv('dataframe2.csv', index=False)
df3.to_csv('dataframe3.csv', index=False)
df4.to_csv('dataframe4.csv', index=False)
df5.to_csv('dataframe5.csv', index=False)

In [ ]:

import pandas as pd
df1 = pd.read_sql_query("""SELECT 
  c.FirstName || " " || c.LastName AS "Clientes",
  ar.Name AS "Nombre artista", ge.Name AS "Género", 
  SUM(il.Quantity * il.UnitPrice) AS "Dinero gastado",
  COUNT(i.InvoiceId) AS "Total de compras",
  TIME(SUM(tr.Milliseconds)/ 1000, 'unixepoch' ) AS "Tiempo de reproducción de pistas"
FROM 
  artists ar 
  JOIN albums al ON ar.ArtistId = al.ArtistId 
  JOIN tracks tr ON al.AlbumId = tr.AlbumId
  JOIN media_types mt ON tr.MediaTypeId = mt.MediaTypeId
  JOIN genres ge ON tr.AlbumId = ge.GenreId  
  JOIN invoice_items il ON tr.TrackId = il.TrackId 
  JOIN invoices i ON il.InvoiceId = i.InvoiceId 
  JOIN customers c ON i.CustomerId = c.CustomerId 
  JOIN employees e ON e.EmployeeId = e.EmployeeId
GROUP BY 
  ar.Name 
ORDER BY 
  SUM(il.Quantity * il.UnitPrice) DESC 
LIMIT 
  10;""", 
                       con=engine.connect())
df1.head()

,Clientes,Nombre artista,Género,Dinero gastado,Total de compras,Tiempo de reproducción de pistas
0,Puja Srivastava,Chico Buarque,Alternative,213.84,216,14:04:03
1,Aaron Mitchell,Chico Science & Nação Zumbi,Opera,198.00,200,11:26:53
2,Richard Cunningham,Caetano Veloso,Comedy,166.32,168,10:33:42
3,Fernanda Ramos,AC/DC,Alternative & Punk,126.72,128,09:26:00
4,François Tremblay,Bruce Dickinson,TV Shows,95.04,96,08:45:21
